# Imports

In [2]:
import os
import gc
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from PIL import Image, ImageDraw, ImageFont

import pickle

from scipy.spatial.transform import Rotation as R
from sklearn.metrics.pairwise import cosine_similarity

import supervision as sv
import open3d as o3d
from cuml.cluster import DBSCAN
import cupy as cp

import concurrent.futures
from collections import Counter

import clip

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import torch
import torchvision.transforms as transforms
from torchvision.ops import box_convert
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

# Setup

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
weights_dir = "/scratch/kumaradi.gupta/checkpoints"

imgs_dir = "/scratch/kumaradi.gupta/run_kinect_wheel_1/rgb"
depth_dir = "/scratch/kumaradi.gupta/run_kinect_wheel_1/depth/"
pose_dir = "/scratch/kumaradi.gupta/run_kinect_wheel_1/pose/"

img_dict_dir = "/scratch/kumaradi.gupta/kinect_img_dict.pkl"

In [6]:
def generate_pastel_color():
    return (
        int(random.uniform(0.5, 1.0) * 255), 
        int(random.uniform(0.5, 1.0) * 255), 
        int(random.uniform(0.5, 1.0) * 255)
    )

In [7]:
# Load from pickle file
with open(img_dict_dir, 'rb') as file:
    img_dict = pickle.load(file)


In [8]:
'''
img_dict = {img_name: {img_path: str,
                        ram_tags: list_of_str,
                        objs: {0: {bbox: [x1, y1, x2, y2],
                                    phrase: str,
                                    clip_embed: [1, 1024]},
                                    dino_embed: [1, 1024]},
                                    mask: [h, w],
                                    prob: float,
                                    aabb: arr}
                                1: {...},
                        }
            img_name: {...},
            }
'''

def get_depth(img_name):
    # depth_path = os.path.join(depth_dir, img_name + '.npy')
    # depth = np.load(depth_path)

    depth_path = os.path.join(depth_dir, img_name + '.png')
    depth = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)
    depth = depth.astype(np.float32) / 1000.0
    return depth

def get_pose(img_name):
    pose_path = os.path.join(pose_dir, img_name + '.txt')

    # check if the pose file exists, if it doesn't, return None
    if not os.path.exists(pose_path):
        return None
    
    with open(pose_path, 'r') as f:
        pose = f.read().split()
        pose = np.array(pose).astype(np.float32)
    return pose

def get_sim_cam_mat_with_fov(h, w, fov):
    cam_mat = np.eye(3)
    cam_mat[0, 0] = cam_mat[1, 1] = w / (2.0 * np.tan(np.deg2rad(fov / 2)))
    cam_mat[0, 2] = w / 2.0
    cam_mat[1, 2] = h / 2.0
    return cam_mat

def get_realsense_cam_mat():
    K = np.array([[386.458, 0, 321.111],
              [0, 386.458, 241.595],
              [0, 0, 1]])
    return K

def get_kinect_cam_mat():
    K = np.array([[9.7096624755859375e+02, 0., 1.0272059326171875e+03], 
                  [0., 9.7109600830078125e+02, 7.7529718017578125e+02], 
                  [0., 0., 1]])
    return K

In [9]:
def create_point_cloud(img_id, obj_data, cam_mat, color=(1, 0, 0), cam_height=0.9):
    """
    Generates a point cloud from a depth image, camera intrinsics, mask, and pose.
    Only points within the mask and with valid depth are added to the cloud.
    Points are colored using the specified color.
    """
    
    depth = get_depth(img_id)
    pose = get_pose(img_id)
    mask = obj_data['mask']

    if pose is None:
        return o3d.geometry.PointCloud()

    # Reproject the depth to 3D space
    rows, cols = np.where(mask)

    depth_values = depth[rows, cols]
    valid_depth_indices = (depth_values > 0) & (depth_values <= 5)

    rows = rows[valid_depth_indices]
    cols = cols[valid_depth_indices]
    depth_values = depth_values[valid_depth_indices]

    points2d = np.vstack([cols, rows, np.ones_like(rows)])

    cam_mat_inv = np.linalg.inv(cam_mat)
    points3d_cam = cam_mat_inv @ points2d * depth_values

    # Parse the pose
    pos = np.array(pose[:3], dtype=float).reshape((3, 1))
    quat = pose[3:]
    rot = R.from_quat(quat).as_matrix()

    # # Apply rotation correction, to match the orientation z: backward, y: upward, and x: right
    # rot_ro_cam = np.eye(3)
    # rot_ro_cam[1, 1] = -1
    # rot_ro_cam[2, 2] = -1
    # rot = rot @ rot_ro_cam

    # # Apply position correction
    # pos[1] += cam_height

    # Create the pose matrix
    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = rot
    pose_matrix[:3, 3] = pos.reshape(-1)

    # Transform the points to global frame
    points3d_homo = np.vstack([points3d_cam, np.ones((1, points3d_cam.shape[1]))])
    points3d_global_homo = pose_matrix @ points3d_homo
    points3d_global = points3d_global_homo[:3, :]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d_global.T)

    # Assign color to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points3d_global.shape[1], 1)))

    return pcd


In [10]:
def fast_DBSCAN(point_cloud_o3d, eps=0.2, min_samples=20):

    if point_cloud_o3d.is_empty():
        return point_cloud_o3d

    # Convert Open3D point cloud to NumPy arrays
    points_np = np.asarray(point_cloud_o3d.points)
    colors_np = np.asarray(point_cloud_o3d.colors)

    # Convert NumPy array to CuPy array for GPU computations
    points_gpu = cp.asarray(points_np)

    # Create a DBSCAN instance with cuML
    dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)

    # Fit the model to the GPU data
    dbscan_model.fit(points_gpu)

    # Get the labels for the clusters
    labels_gpu = dbscan_model.labels_

    # Convert the labels back to a NumPy array
    labels = cp.asnumpy(labels_gpu)

    # Count the occurrence of each label to find the largest cluster
    label_counter = Counter(labels)
    label_counter.pop(-1, None)  # Remove the noise label (-1)
    if not label_counter:  # If all points are noise, return an empty point cloud
        return o3d.geometry.PointCloud()

    # Find the label of the largest cluster
    largest_cluster_label = max(label_counter, key=label_counter.get)

    # Filter the points and colors that belong to the largest cluster
    largest_cluster_points = points_np[labels == largest_cluster_label]
    largest_cluster_colors = colors_np[labels == largest_cluster_label]

    # Create a new Open3D point cloud with the points and colors of the largest cluster
    largest_cluster_point_cloud_o3d = o3d.geometry.PointCloud()
    largest_cluster_point_cloud_o3d.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_point_cloud_o3d.colors = o3d.utility.Vector3dVector(largest_cluster_colors)

    return largest_cluster_point_cloud_o3d


In [11]:
def custom_cosine_similarity(vec1, vec2):
    # Ensure the vectors have the same shape
    if vec1.shape != vec2.shape:
        raise ValueError("Vectors must have the same shape.")

    # Compute the dot product of the vectors
    dot_product = np.dot(vec1, vec2)

    # Compute the magnitudes (Euclidean norms) of the vectors
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)

    # Compute the cosine similarity
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)

    # Normalize the similarity value to [0, 1]
    normalized_similarity = 0.5 * (similarity + 1)

    return normalized_similarity

In [12]:
# Function to convert Open3D point cloud to NumPy array
def pointcloud_to_numpy(pcd):
    return np.asarray(pcd.points)
    # return np.asarray(pcd.points), np.asarray(pcd.colors)

# Function to convert NumPy array to Open3D point cloud
def numpy_to_pointcloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd


In [22]:
# Load the model
model, transform = clip.load("ViT-B/32", device=device)

def get_text_clip_embedding(text):
    text_inputs = clip.tokenize([text]).to(device)
    
    # Get the text features
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        
    # Normalize the features
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    return text_features.cpu().squeeze().numpy()

ceiling_embed = get_text_clip_embedding("This is an image of a ceiling")
wall_embed = get_text_clip_embedding("This is an image of a wall")
floor_embed = get_text_clip_embedding("This is an image of floor")
chair_embed = get_text_clip_embedding("This is an image of a chair")
background_embed = get_text_clip_embedding("This is an image of a ceiling or wall or floor or pillar")

del model
del transform
torch.cuda.empty_cache()
gc.collect()


0

# Obj Nodes Creation

In [13]:
def cosine_sim(clip_embed, node_clip_embed):
    return custom_cosine_similarity(clip_embed, node_clip_embed)

def nnratio(pcd, node_pcd, delta_nn):
    # Use Open3D's KDTree
    kdtree = o3d.geometry.KDTreeFlann(node_pcd)

    count = 0
    for i in range(len(pcd.points)):
        point = pcd.points[i]
        # Find neighbors within a radius
        [return_count, idx, _] = kdtree.search_radius_vector_3d(point, delta_nn)

        # If any neighbors are found within the radius
        if return_count > 0:
            count += 1

    # Proportion of points within distance threshold
    ratio = count / len(pcd.points)

    return ratio


def delta_sim(pcd, node_pcd, clip_embed, node_clip_embed, params):
    if pcd.is_empty() or node_pcd.is_empty():
        return 0

    delta_geo = nnratio(pcd, node_pcd, params['delta_nn'])
    delta_sem = cosine_sim(clip_embed, node_clip_embed)

    delta_sim = delta_geo + delta_sem
    
    return delta_sim


In [14]:
def merge_nodes(node1, node2, params):
    # Merge source IDs: source_ids: [(img_id, obj_id), ...]
    source_ids = node1['source_ids'] + node2['source_ids']
    count = len(source_ids)

    # Average the embeddings
    avg_clip_embed = (np.array(node1['clip_embed']) * len(node1['source_ids']) +
                      np.array(node2['clip_embed']) * len(node2['source_ids'])) / count

    avg_dino_embed = (np.array(node1['dino_embed']) * len(node1['source_ids']) +
                      np.array(node2['dino_embed']) * len(node2['source_ids'])) / count

    # Combine point clouds
    merged_pcd = node1['pcd']
    merged_pcd.points.extend(node2['pcd'].points)
    merged_pcd = merged_pcd.voxel_down_sample(voxel_size=params['voxel_size'])

    # make all points the same color (node1's color)
    merged_pcd.colors = o3d.utility.Vector3dVector(np.tile(node1['pcd'].colors[0], (len(merged_pcd.points), 1)))

    # Concatenate the points contributions from both nodes
    points_contri = node1['points_contri'] + node2['points_contri']

    return {
        'source_ids': source_ids,
        'clip_embed': avg_clip_embed,
        'dino_embed': avg_dino_embed,
        'pcd': merged_pcd,
        'points_contri': points_contri
    }

In [15]:
def get_merge_scene_node_id(similarities, scene_obj_nodes, params):
    # Find the node with the minimum similarity value, greedy assignment
    max_sim_node_id = max(similarities, key=similarities.get)

    # Check if the similarity is below the threshold
    if similarities[max_sim_node_id] >= params['sim_thresh']:
        return max_sim_node_id
    else:
        return None


In [30]:
background_words = ['ceiling', 'wall', 'floor', 'pillar', 'door', 'basement']
background_phrase = ['office']

def check_background(obj_data):
    obj_phrase = obj_data['phrase']
    if obj_phrase in background_phrase:
        return True

    obj_words = obj_phrase.split()
    for word in obj_words:
        if word in background_words:
            return True
    return False


In [31]:
def init_scene_nodes(img_dict, params):
    # Initialize an empty dictionary to store scene object nodes
    scene_obj_nodes = {}

    # Retrieve the initial image data using the provided ID
    img_data = img_dict[params['init_img_id']]
    img_path = img_data['img_path']
    img_id = img_path.split('/')[-1].split('.')[0]

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image from BGR to RGB format

    # Retrieve objects present in the image
    objs = img_data['objs']
    node_count = 1

    for obj_id in objs:
        obj_data = objs[obj_id]

        # Calculate similarities
        check_background_flag = check_background(obj_data)

        if check_background_flag:
            node_id = 0
        else:
            node_id = node_count
            node_count += 1
        
        color = generate_pastel_color()
        # Create a point cloud for the object
        pcd = create_point_cloud(img_id, obj_data, params['cam_mat'], color=color)
        pcd = pcd.voxel_down_sample(voxel_size=params['voxel_size'])

        if node_id != 0:
            pcd = fast_DBSCAN(pcd, eps=params['eps'], min_samples=params['min_samples'])

        if pcd.is_empty():
            continue
        
        if node_id not in scene_obj_nodes:
            # Store the object data in the scene object nodes dictionary
            scene_obj_nodes[node_id] = {'source_ids': [(params['init_img_id'], obj_id)], 
                                        'clip_embed': objs[obj_id]['clip_embed'], 
                                        'dino_embed': objs[obj_id]['dino_embed'], 
                                        'pcd': pcd, 
                                        'points_contri': [len(pcd.points)]}  # Count of points in the point cloud
        else:
            # Merge the object with the existing node
            scene_obj_nodes[node_id] = merge_nodes(scene_obj_nodes[node_id], 
                                                    {'source_ids': [(params['init_img_id'], obj_id)], 
                                                    'clip_embed': objs[obj_id]['clip_embed'], 
                                                    'dino_embed': objs[obj_id]['dino_embed'], 
                                                    'pcd': pcd, 
                                                    'points_contri': [len(pcd.points)]}, params)

    return scene_obj_nodes


In [34]:
def compute_obj_similarity(node_id, node_data, pcd, clip_embed, params):
    # Calculate and return the similarity between the provided point cloud (pcd) and node's point cloud
    return node_id, delta_sim(pcd, node_data['pcd'], clip_embed, node_data['clip_embed'], params)

def process_object(img_id, obj_data, scene_obj_nodes, params):

    color = generate_pastel_color()
    obj_embed = obj_data['clip_embed']

    check_background_flag = check_background(obj_data)
    if check_background_flag:
        node_id = 0

    # Create a point cloud for the object
    pcd = create_point_cloud(img_id, obj_data, params['cam_mat'], color=color)
    pcd = pcd.voxel_down_sample(voxel_size=params['voxel_size'])

    if node_id != 0:
        pcd = fast_DBSCAN(pcd, eps=params['eps'], min_samples=params['min_samples'])

    if pcd.is_empty():
        return None, None
    
    # Compute similarities between the object and all nodes in the scene
    similarities = {}

    if node_id == 0:
        # Manually set the similarity for node 0 as 2, and for all other nodes as 0
        for key in scene_obj_nodes.keys():
            similarities[key] = 2 if key == '0' else 0
    else:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_sims = {executor.submit(compute_obj_similarity, node_id, node_data, pcd, obj_data['clip_embed'], params): node_id for node_id, node_data in scene_obj_nodes.items()}
            for future in concurrent.futures.as_completed(future_sims):
                node_id = future_sims[future]
                similarities[node_id] = future.result()[1]

        
    return similarities, pcd

In [35]:
def update_scene_nodes(img_id, img_data, scene_obj_nodes, params):

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for obj_id, obj_data in img_data['objs'].items():
            # Start asynchronous processing for each object
            future = executor.submit(process_object, img_id, obj_data, scene_obj_nodes, params)
            futures.append((obj_id, img_id, future))

        for obj_id, current_img_id, future in futures:
            similarities, obj_pcd = future.result()

            if similarities is None:
                continue
            
            # Determine whether to merge the object with an existing node or create a new one (id or None)
            merge_scene_node_id = get_merge_scene_node_id(similarities, scene_obj_nodes, params)

            if merge_scene_node_id is not None:
                new_node = {
                    'source_ids': [(obj_id, current_img_id)],
                    'clip_embed': img_data['objs'][obj_id]['clip_embed'],
                    'dino_embed': img_data['objs'][obj_id]['dino_embed'],
                    'pcd': obj_pcd,
                    'points_contri': [len(obj_pcd.points)]}
                
                scene_node = scene_obj_nodes[merge_scene_node_id]
                
                scene_obj_nodes[merge_scene_node_id] = merge_nodes(scene_node, new_node, params)
            else:
                new_node_id = (max([int(i) for i in scene_obj_nodes.keys()]) + 1)
                scene_obj_nodes[new_node_id] = {
                    'source_ids': [(obj_id, current_img_id)],
                    'clip_embed': img_data['objs'][obj_id]['clip_embed'],
                    'dino_embed': img_data['objs'][obj_id]['dino_embed'],
                    'pcd': obj_pcd,
                    'points_contri': [len(obj_pcd.points)]
                }

    return scene_obj_nodes

In [36]:
def compute_node_similarity(node1_id, node1_data, node2_id, node2_data, params):
    # Computes the similarity between two nodes.
    similarity = delta_sim(node1_data['pcd'], node2_data['pcd'], 
                           node1_data['clip_embed'], node2_data['clip_embed'], params)
    return node1_id, node2_id, similarity

def merge_similar_nodes(scene_obj_nodes, params):
    # Merges nodes in the scene that have high similarity.
    nodes_to_remove = set()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        
        # Create a list of node pairs to check
        node_ids = list(scene_obj_nodes.keys())
        # remove node_id 0 from the list as it has ceiling, wall, floor
        node_ids.remove(0)
        
        for i in range(len(node_ids)):
            for j in range(i+1, len(node_ids)):
                node1_id = node_ids[i]
                node2_id = node_ids[j]

                # Avoid re-checking nodes that have already been merged
                if node1_id in nodes_to_remove or node2_id in nodes_to_remove:
                    continue

                future = executor.submit(compute_node_similarity, node1_id, scene_obj_nodes[node1_id],
                                         node2_id, scene_obj_nodes[node2_id], params)
                futures.append(future)
        
        # Process the computed similarities and merge nodes if needed
        for future in concurrent.futures.as_completed(futures):
            node1_id, node2_id, similarity = future.result()
            
            if similarity >= params['sim_thresh']:
                merged_node = merge_nodes(scene_obj_nodes[node1_id], scene_obj_nodes[node2_id])
                scene_obj_nodes[node1_id] = merged_node
                
                # Mark the second node for removal after merging
                nodes_to_remove.add(node2_id)

    # Remove nodes that were merged into other nodes
    for node_id in nodes_to_remove:
        del scene_obj_nodes[node_id]

    return scene_obj_nodes


In [37]:
params = {'init_img_id': '52',
          'sim_thresh': 1.1,
          'voxel_size': 0.025,
          'eps': 0.075,
          'min_samples': 10,
          'delta_nn': 0.025,
          'cam_mat': get_kinect_cam_mat(),
          'clip_sim_thresh': 0.9}

In [51]:
scene_obj_nodes = init_scene_nodes(img_dict, params)

In [52]:
# for id in scene_obj_nodes.keys():
#     print(id, scene_obj_nodes[id]['source_ids'])

for node_id, node_data in scene_obj_nodes.items():
    print(node_id, len(node_data['pcd'].points))
    o3d.io.write_point_cloud(f'/scratch/kumaradi.gupta/kinect_pcds/{node_id}.pcd', node_data['pcd'])

1 1094
2 650
0 9722
4 192
6 541


In [23]:
scene_obj_nodes = merge_similar_nodes(scene_obj_nodes, params)

In [34]:
counter = 0
for img_id, img_data in tqdm(img_dict.items()):
    if len(img_data['objs']) == 0 or img_id == params['init_img_id']:
        continue

    scene_obj_nodes = update_scene_nodes(img_id, img_data, scene_obj_nodes, params)
    
    counter += 1
    if counter % 100 == 0:
        scene_obj_nodes = merge_similar_nodes(scene_obj_nodes, params)


print("Number of nodes in the scene: ", len(scene_obj_nodes))
for node_id, node_data in scene_obj_nodes.items():
    o3d.io.write_point_cloud(f'/scratch/kumaradi.gupta/kinect_pcds/{node_id}.pcd', node_data['pcd'])

  0%|          | 0/20 [00:00<?, ?it/s]

[W] [16:21:17.768880] Batch size limited by the chosen integer type (4 bytes). 35927 -> 28553. Using the larger integer type might result in better performance
[W] [16:21:30.026508] Batch size limited by the chosen integer type (4 bytes). 30262 -> 24052. Using the larger integer type might result in better performance
[W] [16:21:54.901034] Batch size limited by the chosen integer type (4 bytes). 46183 -> 36704. Using the larger integer type might result in better performance
[W] [16:22:15.207469] Batch size limited by the chosen integer type (4 bytes). 34253 -> 27222. Using the larger integer type might result in better performance
[W] [16:22:23.066749] Batch size limited by the chosen integer type (4 bytes). 34124 -> 27120. Using the larger integer type might result in better performance
[W] [16:22:51.891188] Batch size limited by the chosen integer type (4 bytes). 38720 -> 30773. Using the larger integer type might result in better performance
[W] [16:23:07.680593] Batch size limited